# THX Sound

Trying to recreate the THX sound, inspired by this thing:

![](THX_score.png)

It doesn't really hit the mark, but it's satisfyingly close to the right sound.  Other things that could improve it:

* The voices in the original seem to have some lowpass filter sweep.
* It sounds like there are real strings in the original.
* I cheat and fade in the frequency sweep to the sustained frequencies at the end to avoid phase issues transitioning from a chirp to a single frequency for each voice
* As seen in the image and heard in the original, each voice kind of dances around - they're not on a single curved path like mine.
* The notes and balance aren't _quite_ right and the balance between each voice is more bass-heavy in the original

In [1]:
import numpy as np
from IPython.display import Audio

In [2]:
import sys
sys.path.append('../musimathics/')

from pitches import frequencies_for_note_labels

In [3]:
from scipy.signal import chirp
from scipy.signal import sawtooth

num_voices = 29

# some voices in the real sound definitely start in much higher than 400 Hz
starting_frequencies = np.random.uniform(100, 1000, num_voices)

ending_semitones_relative_to_a = np.array(np.arange(num_voices))
bass_notes = np.asarray(frequencies_for_note_labels(['D0', 'D1', 'A2', 'D2'] * 2)) # 2 voices per note in the bass
high_notes = np.asarray(frequencies_for_note_labels(['A3', 'D3', 'A4', 'D4', 'A5', 'D5', 'F#5'] * 3))
high_notes += np.random.uniform(-3, 2, high_notes.size) # slightly detuned
ending_frequencies = np.append(bass_notes, high_notes)

velocity_curve = 2 * np.logspace(-10, 10) # ~20dB range

total_duration_sec = 34
frequency_convergence_sec = 18
fade_out_sec = 1
fade_in_sec = 2

sample_rate = 44100
t_conv = np.linspace(0.0, frequency_convergence_sec, frequency_convergence_sec * sample_rate)
t_rest = np.linspace(0, (total_duration_sec - frequency_convergence_sec), (total_duration_sec - frequency_convergence_sec) * sample_rate)
t_all = np.linspace(0.0, total_duration_sec, total_duration_sec * sample_rate)

summed_sines = np.zeros(t_conv.size + t_rest.size - fade_in_sec * sample_rate)
for voice_index in range(num_voices):
    starting_frequency = starting_frequencies[voice_index]
    ending_frequency = ending_frequencies[voice_index]

    offset = np.random.randint(-sample_rate / 2, sample_rate / 2)
    # sawtooths for upper voices and sines for low
    gen = sawtooth if voice_index > bass_notes.size else np.cos
    sine = gen(2 * np.pi * t_all[:t_conv.size + offset] / 2 * np.logspace(np.log10(starting_frequency), np.log10(ending_frequency), t_conv.size + offset) +  np.random.uniform(0, np.pi))
    sine[-fade_out_sec * sample_rate:] *= np.linspace(1, 0, fade_out_sec * sample_rate)

    summed_sines[:t_conv.size + offset] += sine
    sine = gen(2 * np.pi * t_rest * ending_frequency + np.random.uniform(0, np.pi))
    sine[:fade_in_sec * sample_rate] *= np.linspace(0, 1, fade_in_sec * sample_rate)

    summed_sines[t_conv.size -fade_in_sec * sample_rate:] += sine

summed_sines[:t_conv.size] *= np.logspace(-3.5, 0, t_conv.size)
summed_sines[-fade_out_sec * sample_rate:] *= np.logspace(0, -10, fade_out_sec * sample_rate)
summed_sines /= summed_sines.max()

Audio(summed_sines, rate=sample_rate)

In [4]:
from scipy.io.wavfile import write as wavwrite

In [5]:
wavwrite(filename='THX.wav', data=summed_sines, rate=sample_rate)